In [1]:
import torch
from torch import nn

In [4]:
inputs=torch.Tensor([[[0.2,0.1,0.3],[0.5,0.1,0.1]]])
B,S,E=inputs.shape
inputs=inputs.view(S,B,E)
print(inputs.shape)

torch.Size([2, 1, 3])


In [5]:
parameter_shape=inputs.shape[-2:]
gama=nn.Parameter(torch.ones(parameter_shape))
beta=nn.Parameter(torch.zeros(parameter_shape))
gama.shape,beta.shape

(torch.Size([1, 3]), torch.Size([1, 3]))

In [8]:
dims=[-(i+1) for i in range(len(parameter_shape))]
dims

[-1, -2]

In [43]:
mean=inputs.mean(dims,keepdim=True)
mean.shape

torch.Size([2, 1, 1])

In [46]:
mean

tensor([[[0.2000]],

        [[0.2333]]])

In [53]:
var=((inputs-mean)**2).mean(dims,keepdim=True)
epsilon=1e-5
std=(var+epsilon).sqrt()
std

tensor([[[0.0817]],

        [[0.1886]]])

In [54]:
y=(inputs-mean)/std
y

tensor([[[ 0.0000, -1.2238,  1.2238]],

        [[ 1.4140, -0.7070, -0.7070]]])

In [55]:
out= gama*y+beta
out

tensor([[[ 0.0000, -1.2238,  1.2238]],

        [[ 1.4140, -0.7070, -0.7070]]], grad_fn=<AddBackward0>)

# Class

In [72]:
import torch
from torch import nn

class LayerNormalization():
    def __init__(self,parameter_shape,eps=1e-5):
        self.parameter_shape=parameter_shape
        self.eps=eps
        self.gama=nn.Parameter(torch.ones(parameter_shape))
        self.beta=nn.Parameter(torch.zeros(parameter_shape))

    def forward(self,x):
        
        dims=[-(i+1) for i in range(len(self.parameter_shape))]
        mean=x.mean(dims,keepdim=True)
        print(f"Mean \n ({mean.size()}): \n {mean}")
        var=((x-mean)**2).mean(dims,keepdim=True)
        std=(var+self.eps).sqrt()
        print(f"Standard Deviation \n ({std.size()}): \n {std}")
        y=(x-mean)/std
        print(f"y \n ({y.size()}) = \n {y}")
        out=self.gama*y+self.beta
        print(f"out \n ({out.size()}) = \n {out}")
        return out

In [73]:
batch_size=3
sentence_length=5
embedding_size=8
inputs=torch.randn(batch_size,sentence_length,embedding_size)
print(f"input \n ({inputs.size()}) = \n {inputs}")


input 
 (torch.Size([3, 5, 8])) = 
 tensor([[[ 1.7863,  1.6969, -0.0267, -1.1018, -0.4357,  1.0984,  0.7715,
           0.5637],
         [-0.3976,  0.0082, -0.7245, -1.9276,  0.1923,  0.9856,  1.1037,
           1.0151],
         [ 1.1660, -0.5499,  0.5647, -0.4880, -0.7653, -1.4199, -0.5597,
          -0.1057],
         [ 2.5553, -0.1269, -1.1339, -0.1698,  0.8197,  0.5151,  0.5285,
          -0.5106],
         [ 0.2514, -0.0576,  2.5786, -0.0999,  0.1893,  1.1774,  0.6163,
          -1.0078]],

        [[ 1.1933,  1.6976, -1.1990,  0.3429, -0.4628,  0.0199, -2.4676,
           1.0150],
         [ 0.8992,  0.4890,  0.3794,  0.8350, -1.8216, -1.1510, -0.0731,
           0.3796],
         [ 0.3672,  1.0593, -1.1047, -0.3547,  0.0856,  0.1348,  0.5807,
          -0.4097],
         [-1.7095, -0.0088, -1.3868,  2.0350,  0.4451, -0.6855, -0.2023,
           0.6463],
         [ 0.0920, -0.3740,  2.1329, -0.6932, -0.3044,  0.4330, -2.2930,
          -0.3805]],

        [[-0.4710, -0.4528, -1

In [74]:
inputs.shape[-1:]

torch.Size([8])

In [75]:
layer_norm=LayerNormalization(inputs.shape[-1:])

In [76]:
out=layer_norm.forward(inputs)

Mean 
 (torch.Size([3, 5, 1])): 
 tensor([[[ 0.5441],
         [ 0.0319],
         [-0.2697],
         [ 0.3097],
         [ 0.4560]],

        [[ 0.0174],
         [-0.0079],
         [ 0.0448],
         [-0.1083],
         [-0.1734]],

        [[ 0.0173],
         [-0.4254],
         [-0.6360],
         [ 0.3123],
         [ 0.0433]]])
Standard Deviation 
 (torch.Size([3, 5, 1])): 
 tensor([[[0.9504],
         [0.9777],
         [0.7553],
         [1.0350],
         [0.9925]],

        [[1.2842],
         [0.9138],
         [0.6243],
         [1.1191],
         [1.1522]],

        [[0.8033],
         [0.6649],
         [0.9391],
         [0.8915],
         [0.7891]]])
y 
 (torch.Size([3, 5, 8])) = 
 tensor([[[ 1.3070e+00,  1.2130e+00, -6.0059e-01, -1.7317e+00, -1.0309e+00,
           5.8325e-01,  2.3930e-01,  2.0650e-02],
         [-4.3932e-01, -2.4190e-02, -7.7360e-01, -2.0042e+00,  1.6405e-01,
           9.7542e-01,  1.0962e+00,  1.0056e+00],
         [ 1.9010e+00, -3.7099e-01,  1.

In [77]:
out[0].mean(),out[0].std()   

(tensor(-2.6822e-08, grad_fn=<MeanBackward0>),
 tensor(1.0127, grad_fn=<StdBackward0>))